In [ ]:
import astropy.coordinates as coord
from astropy import units as u
from astroquery.simbad import Simbad
import pandas as pd  
import numpy as np
import time

In [1]:
import astropy.coordinates as coord
from astropy import units as u
from astroquery.simbad import Simbad
import pandas as pd  
import numpy as np
import time

In [37]:
from astroquery.irsa import Irsa



In [15]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
dfhiq=pd.read_csv("DATA/rms_high_qf.csv")
df6 = pd.read_csv('DATA/MSX6C.csv',sep=";", skiprows=range(0,88),names = ["Name","RAJ20(deg)","DEJ20(deg)",
                                                  "F4_B1(Jy)","qF4_B1","eF4_B1(%)","F8","qF8","eF8(%)",
                                                                                    "F12","qF12","eF12(%)",
                                                                                     "F14","qF14","eF14(%)",
                                                                                    "F21","qF21","eF21(%)"])
dfnrms=df6[~np.in1d(df6['Name'],dfhiq['MSX Name'])]

In [7]:
dfnrms=df6[~np.in1d(df6['Name'],dfhiq['MSX Name'])]

In [8]:
dfnrms.head()

,Name,RAJ20(deg),DEJ20(deg),F4_B1(Jy),qF4_B1,eF4_B1(%),F8,qF8,eF8(%),F12,qF12,eF12(%),F14,qF14,eF14(%),F21,qF21,eF21(%)
0,G286.8386-02.1157,158.4852,-60.6174,-11.57,0,,20.02,4,4.1,22.54,4,5.0,17.73,4,6.1,17.720,4,6.0
1,G326.2281-02.1979,238.6553,-56.5447,15.95,1,13.1,18.39,4,4.1,17.37,4,5.0,12.43,4,6.1,11.450,4,6.1
2,G319.0885-02.1148,227.3443,-60.5847,27.92,2,11.2,17.78,4,4.1,23.58,4,5.0,17.09,4,6.1,10.880,4,6.1
3,G005.4484-02.0026,271.4327,-25.2398,22.57,1,12.3,16.67,4,4.1,19.36,4,5.0,14.77,4,6.1,10.580,4,6.1
4,G313.6091-02.2572,217.0207,-63.1080,24.18,1,11.1,18.46,4,4.1,17.27,4,5.0,11.75,4,6.1,7.952,4,6.1


In [10]:
len(dfnrms),len(df6)

(16912, 20718)

In [11]:
lon=[float(i[1:8]) for i in dfnrms['Name']]
lonmsx=np.array(lon)
lat=[float(i[9:17]) for i in dfnrms['Name']]
latmsx=np.array(lat)

In [12]:
customSimbad = Simbad()
customSimbad.add_votable_fields('otype')
#customSimbad._VOTABLE_FIELDS.append('otypes')
customSimbad.add_votable_fields('bibcodelist(1999-2019)')

In [25]:
result=[]
for i,j in zip(range(len(lonmsx)),dfnrms['Name']):
    if i%100 == 0:
        print(i)
        time.sleep(0.1)
    try:
        result_table = customSimbad.query_region(coord.SkyCoord(lonmsx[i], latmsx[i],unit=(u.deg, u.deg), frame='galactic'),radius=5 * u.arcsec)
        result+=[result_table]
    except:
        print(i,j)
        time.sleep(2)
        result_table = customSimbad.query_region(coord.SkyCoord(lonmsx[i], latmsx[i],unit=(u.deg, u.deg), frame='galactic'),radius=5 * u.arcsec)
        result+=[result_table]
        print(result_table)

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500
11600
11700
11800
11900
12000
12100
12200
12300
12400
12500
12600
12700
12800
12900
13000
13100
13200
13300
13400
13500
13600
13700
13800
13900
14000
14100
14200
14300
14400
14500
14600
14700
14800
14900
15000
15100
15200
15300
15400
15500
15600
15700
15800
15900
16000
16100
16200
16300
16400
16500
16600
16700
16800
16900


In [26]:
np.sum([np.sum(i==None) for i in result])

14077

In [27]:
olist=[]
nlist=[]
nom=0
for i,j in zip(result,dfhiq['MSX Name']):
    try:
        olist+=[str(i['OTYPE'][i['BIBLIST_1999_2019']>=10][0],'UTF')]
        nlist+=[j]
    except:
        nom+=1

In [28]:
dfxsi = pd.DataFrame(
    {'MSX Name': nlist,
     'Source Type': olist,
    })

In [29]:
dfxsi.to_csv('DATA/MSX_SIMBAD_OTYPE.csv')

In [30]:
len(olist)

148

In [32]:
np.unique(olist)

array(['**', 'Ae*', 'Be*', 'BlueSG*', 'C*', 'Candidate_C*',
       'Candidate_LP*', 'Candidate_WR*', 'EB*betLyr', 'Em*', 'HH', 'HII',
       'IR', 'LPV*', 'Mira', 'Nova', 'OH/IR', 'PM*', 'PN', 'PulsV*RVTau',
       'RedSG*', 'S*', 'SB*', 'SG*', 'Star', 'Symbiotic*', 'V*', 'WR*',
       'YellowSG*', 'denseCore', 'gamma', 'post-AGB*'], dtype='<U13')

In [33]:
len(dfxsi)

148

In [95]:
dfhiq=pd.read_csv("DATA/rms_high_qf.csv")
df6 = pd.read_csv('DATA/MSX6C.csv',sep=";", skiprows=range(0,88),names = ["Name","RAJ20(deg)","DEJ20(deg)",
                                                "F4_B1(Jy)","qF4_B1","eF4_B1(%)","F8","qF8","eF8(%)",
                                                                                  "F12","qF12","eF12(%)",
                                                                                   "F14","qF14","eF14(%)",
                                                                                  "F21","qF21","eF21(%)"])
dfnrms=df6[~np.in1d(df6['Name'],dfhiq['MSX Name'])]


cond_F8 = dfnrms["F8"] > 0
cond_F12 = dfnrms["F12"] > 0
cond_F14 = dfnrms["F14"] > 0
cond_F21 = dfnrms["F21"] > 0

dfnew = dfnrms[cond_F8 & cond_F12 & cond_F14 & cond_F21]

lon_q=[float(i[1:8]) for i in dfnew['Name']]
lon_q=np.array(lon_q)
lat_q=[float(i[9:16]) for i in dfnew['Name']]
lat_q=np.array(lat_q)


result_70 = []
names       = []
Lat       = []

for i,j in zip(range(len(lon_q))[1868:],dfnew['Name'][1868:]): # should be dfnew (1868,5570)
    if i%100 == 0:
        print(i)
#        time.sleep(0.1)
    try:
        result_table = Irsa.query_region(coord.SkyCoord(lon_q[i], lat_q[i],unit=(u.deg, u.deg),frame='galactic'),
                                        catalog='higal70',selcols="glon,glat,fint,err_fint", radius=5 * u.arcsec)
        result_70+=[result_table]
        names+=[j]
    except:
        print(i,j)
        time.sleep(2)

        result_table = Irsa.query_region(coord.SkyCoord(lon_q[i], lat_q[i],unit=(u.deg, u.deg),frame='galactic'),
                                            catalog='higal70',selcols="glon,glat,fint,err_fint", radius=5 * u.arcsec)
        result_70+=[result_table]
        names+=[j]

1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5595 G045.4543+00.0600
5600
5610 G011.2183+00.1265


ConnectTimeout: HTTPSConnectionPool(host='irsa.ipac.caltech.edu', port=443): Max retries exceeded with url: /cgi-bin/Gator/nph-query?radunits=arcsec&outrows=500&outfmt=3&catalog=higal70&selcols=glon%2Cglat%2Cfint%2Cerr_fint&radius=5.0&spatial=Cone&objstr=272.4506809308934+-19.16366025002509 (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x130bedbe0>, 'Connection to irsa.ipac.caltech.edu timed out. (connect timeout=60)'))

In [77]:
dataoutnrms = open("Flux_70_nrms.csv","w")
dataoutnrms.write("%s,%s\n"%("Name","Flux_70")) # 1868

13

In [78]:
len(result_70)

3727

In [83]:
flux_70 = []
for i in zip(result_70,names):
    try:
        if i["fint"][0] < 200:
            flux_70 += [i["fint"][0]]
            print('hi')
        dataoutnrms.write("%s,%f\n"%(j,i["fint"][0]))
    except:
        if len(i) == 0:
            print("Value not useful")

dataoutnrms.close()

In [87]:
np.sum([len(i) for i in result_70]) # empty for the first 1868 objects

1310

In [97]:
for i in zip(result_70,names):
    if len(i)>0:
        print(i["fint"])
        


TypeError: tuple indices must be integers or slices, not str

In [93]:
dfms=pd.read_csv('Flux_70_nrms.csv')

In [94]:
len(dfms)

0

In [90]:
dfms.head()

,Name,Flux_70
0,G001.0388-02.2297,22.724
1,G357.7959-02.0178,4.866
2,G350.5266-02.2585,24.632
3,G000.6792-02.1680,27.439
4,G353.8934-02.2371,241.931


In [91]:
len(dfnew)

16687